# Datos sin limpiar

In [1]:
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#Data from text file to python list
text_file='cpaa.txt'
f = open(text_file, encoding='utf8')
data=[]
for x in f:
    data.append(x.lower())

In [ ]:
data

In [4]:
#stores the labels in the order they appear, not all articles have all types of data
datalabels=[]
for i in range(len(data)):
    if data[i][0] != ' ': #if the element starts with a space the element contains no label
        if data[i][0:2] in datalabels: #if the label is already registered then the label is stored as the previous label to the next one
            lastlabel=data[i][0:2]
        else:
            if len(datalabels) == 0: #the first label must be registered differently since the loop created do not cover the case were there is no previous label
                datalabels.append(data[i][0:2])
            else: #the label is registered in the index right after the previous label
                for j in range(len(datalabels)):
                    if datalabels[j] == lastlabel:
                        datalabels.insert(j+1,data[i][0:2])
            lastlabel=data[i][0:2]

#things that are not labels, but appear as labels to the for cycle, must be removed
datalabels.remove('\ufefff')
datalabels.remove('\n')
datalabels.remove('vr')

In [5]:
reordered_datalabels=['af','pu','pi','de','id','wc','sc','c1','au']

In [6]:
for x in reordered_datalabels:
    datalabels.remove(x)
    datalabels.append(x)

In [7]:
#A dictionary is created to store the data in their respective labels
datadictionary = {}
for x in datalabels:
    datadictionary[x]=[]

In [ ]:
#Elements that are irrelevant are removed
data.remove('\n')
data.remove('\ufefffn clarivate analytics web of science\n')
data.remove('vr 1.0\n')
n=0

for x in data:
    if x[0:2] == 'pt': #The articles are counted
        n+=1
    if x[0:2] == 'ba': #A spaceial case's index is found
        if n == 6807:    
            print(x)
print(n)

In [9]:
#The data dictionary is filled with the information from data
iodata=-1
currentdatalabel=0
for x in data:
    if x[0:2] == 'pt': #All articles have publication type (pt) as their first element
        iodata+=1
        for y in datalabels:
            datadictionary[y].append(0)#A 0 is added to all lists in the dictionary that corresponds to the article that is identified by the current pt
        datadictionary[x[0:2]][iodata]=x[3:-1]#The 0 that corresponds to the pt list is raplced with the publication type of the current article 
    else:
        if not x[0:2] in datalabels:#if the element does not have a label it correspond to the last datalabel found in an element
            if type(datadictionary[currentdatalabel][iodata]) != str: #if the current dictionary entry is a list the current element is added to that list
                datadictionary[currentdatalabel][iodata].append(x[3:-1])
            else: #if the current dictionary entry is a not a lis, it is converted into one and the new element is added
                datadictionary[currentdatalabel][iodata]=[datadictionary[currentdatalabel][iodata]]
                datadictionary[currentdatalabel][iodata].append(x[3:-1])
        else: #if the element has a label the it is added to the current dictionary entry and the label is marked as the current label
            datadictionary[x[0:2]][iodata]=x[3:-1]
            currentdatalabel=x[0:2]

In [ ]:
#A new dictionary is created to focus on the authors not the articles, so if an article has 5 coauthors 5 copies of the article are saved, one for each author
fddic = {}
for x in datalabels:
    fddic[x]=[]
xn=0
for x in datadictionary['af']:
    aun=0
    if type(x) == str: #if the article has only one author the information is left as is except for the field 'au'
        for z in datalabels:
            if z == 'af':
                fddic[z].append('{}'.format(str(aun))+x)
            elif z == 'au': #Since we have the author's full name in af au is used to save all coauthors full names, in the case of one author this field is 0
                fddic[z].append(0)
            else: #The other fiels are left as is
                fddic[z].append(datadictionary[z][xn])
    else:
        try:
            for y in x: #A copy of the information is generated in the dictionary for each coauthor
                for z in datalabels:
                    if z == 'af': #One of the coauthors is saved as the owner of this copy of the articles information
                        fddic[z].append('{}'.format(str(aun))+y)#The number of coauthor is added to the begining of the name
                    elif z == 'au': #All coauthors are saved as in this field
                        fddic[z].append(x)
                    else: #The other fiels are left as is
                        fddic[z].append(datadictionary[z][xn])
                aun+=1
        except Exception: #The special case metioned above, we can eliminate it because is only one
            print('Uno no se pudo')
    xn+=1

In [ ]:
df = pd.DataFrame(fddic) 
df

In [ ]:
numofcol=0
for x in df.columns:
    if df[x][df[x]==0].count() > 15000: #if a field is 0 for more than half of the authors the field is deleted
        del df[x]
    else:
        numofcol+=1
        print(x,df[x][df[x]==0].count())
print('fields left:',numofcol)

In [13]:
#Fields are eliminated based on their usefulness for AND
for x in ['pt','ti','la','dt','ab','rp','u1','u2','pa','sn','ei','ji','vl','bp','ep','di','pg','we','ga','ut','da','er','c3','nr','tc','z9','cr','fx','pd','py','so','fu','j9']:
    del df[x]

In [ ]:
#The fields we are lef with
for x in df.columns:
    print(x,df[x][df[x]==0].count())

In [15]:
#Elements of the dataframe are saved as copies, so they don't alter one antoher
for x in df.columns:
    for y in df.index:
        try:
            df[x][y]=df[x][y].copy()
        except Exception:
            pass

In [16]:
#Authors are deleted from their goup of coauthors
for x in df.index:
    if type(df['au'][x]) != int:
        try:
            df['au'][x].remove(df['af'][x][1:])
        except Exception:
            df['au'][x].remove(df['af'][x][2:])

In [17]:
#Information might be separated in diferent elements of the same list
#All elements are combined and separated by how they are separated in the source, by a semicolon
def join_split(col):
    c=0
    for x in df[col]:
        if type(x) == list:
            y=' '.join(x.copy())
            df[col][c]=y.split(";")
        elif type(x) == str:
            y=x
            df[col][c]=y.split(";")
        c+=1

In [18]:
join_split('de')

In [19]:
join_split('id')

In [20]:
#C1 refers to the addresses of the authors
#The address is accompanied by a list of the authors whose the address it belongs to
for x in df.index:
    sc=0
    if type(df['c1'][x]) == str: #if it is a string there is only one address
        if df['af'][x][2:] in df['c1'][x]: #It must be verified if the address corresponds to the curent author
            num=2
            for y in df['c1'][x]: #The list of authors is eliminated
                if y != ']':
                    num+=1
                else:
                    df['c1'][x]=df['c1'][x][num:]
        else: #if the author is not in the list the address is left as 0 for the author in question
            df['c1'][x]=0
    elif type(df['c1'][x]) == list: #if there is a list of adresses then each one must be verified 
        for y in df['c1'][x]:
            if df['af'][x][2:] in y:
                num=2
                for z in y:
                    if z != ']':
                        num+=1
                    else:
                        df['c1'][x]=y[num:]
                        sc=1
            else:
                if sc != 1:
                    df['c1'][x]=0

In [21]:
#The emails of the authors appear as a list witho only the orther they appear in as an indicator of what email belongs to who
#Only when we have the same amount of emails as authors we can be certain of what email belongs to who
col='em'
a=0

join_split(col)

for c in df.index:
    if df[col][c] != 0 and df['au'][c] != 0:#First we must have information to analyze
        if len(df['au'][c])+1 == len(df[col][c]):#We verify if there are the same number of emails and authors
            try:
                df[col][c]=df[col][c][int(df['af'][c][0:2])].replace(' ','')#Since the number of coauthor was saved with the name we use that number
            except Exception:
                df[col][c]=df[col][c][int(df['af'][c][0:1])].replace(' ','')#the number might have to digits
        else:
            df[col][c]=0
for i in df.index:
    if type(df['em'][i]) == list:
        df['em'][i]=df['em'][i][0].replace(' ','')
for x in df.index:
    if df[col][x] != 0:
        df[col][x]=df[col][x].split('@')#The emails are splited because it will be usefull

In [22]:
#In the case of this identification number we have the name of the author to identify it
col='ri'
c=0

join_split(col)

for c in df.index:
    if df[col][c] != 0 and df['au'][c] != 0:
        sc=0
        for i in df[col][c]:
            if df['af'][c][3:] in i:
                num=1
                for z in i:
                    if z != '/':
                        num+=1
                    else:
                        df[col][c]=i[num:]
                        sc=1
            else:
                if sc != 1:
                    df[col][c]=0
    if df[col][c] != 0 and df['au'][c] == 0:
        sc=0
        if df['af'][c][3:] in df[col][c]:
            num=1
            for z in i:
                if z != '/':
                    num+=1
                else:
                    df[col][c]=i[num:]
                    sc=1
        else:
            if sc != 1:
                df[col][c]=0

In [23]:
#In the case of this identification number we have the name of the author to identify it
col='oi'
c=0

join_split(col)

for c in df.index:
    if df[col][c] != 0 and df['au'][c] != 0:
        sc=0
        for i in df[col][c]:
            if df['af'][c][3:] in i:
                num=1
                for z in i:
                    if z != '/':
                        num+=1
                    else:
                        df[col][c]=i[num:]
                        sc=1
            else:
                if sc != 1:
                    df[col][c]=0
    if df[col][c] != 0 and df['au'][c] == 0:
        sc=0
        if df['af'][c][3:] in df[col][c]:
            num=1
            for z in i:
                if z != '/':
                    num+=1
                else:
                    df[col][c]=i[num:]
                    sc=1
        else:
            if sc != 1:
                df[col][c]=0

In [24]:
col='wc'
join_split(col)

In [25]:
col='sc'
join_split(col)

In [26]:
for c in df.index:
    df['af'][c]=df['af'][c][1:]
    try:
        int(df['af'][c][0])
        df['af'][c]=df['af'][c][1:]
    except Exception:
        pass

In [ ]:
for x in df.columns:
    print(x,df[x][df[x]==0].count())

In [ ]:
for y in df.index:
    for x in df.columns:
        print(x,df[x][y])
    print('--------------------------------------------------')

In [ ]:
df

# Datos limpios

In [ ]:
ndf1=df.copy()
ndf1 = ndf1.drop(ndf1[(ndf1['em'] == 0) & (ndf1['ri'] == 0) & (ndf1['oi'] == 0)].index)

ndf1

In [ ]:
for y in ndf1.index:
    for x in ndf1.columns:
        print(x,ndf1[x][y])
    print('--------------------------------------------------')

In [ ]:
col1='af'
col2='em'
col3='ri'
col4='oi'
list1=[]
rep=[]
misnomdifa=0
tot=0
datafnw=[]
for x in ndf1.index:
    if not ndf1[col1][x] in list1:
        n=ndf1[col1][ndf1[col1]==ndf1[col1][x]].count()
        list1.append(ndf1[col1][x])
        rep.append(n)
        e=0
        r=0
        o=0
        for y in ndf1[col1][ndf1[col1]==ndf1[col1][x]].index:
            if ndf1[col2][x] != 0 and ndf1[col2][y] != 0:
                if ndf1[col2][x][1] == ndf1[col2][y][1]:
                    if ndf1[col2][x][0] != ndf1[col2][y][0]:
                        e+=1
                        print(ndf1[col2][x],ndf1[col2][y])
            if ndf1[col3][x] != 0 and ndf1[col3][y] != 0:
                if ndf1[col3][x] != ndf1[col3][y]:
                    r+=1
                    print(ndf1[col3][x],ndf1[col3][y])
            if ndf1[col4][x] != 0 and ndf1[col4][y] != 0:
                if ndf1[col4][x] != ndf1[col4][y]:
                    o+=1
                    print(ndf1[col4][x],ndf1[col4][y])
    if e != 0 or r != 0 or o != 0:
        print(ndf1[col1][x],n)
        print('H:',e,r,o)
        misnomdifa+=1
        print('----------------------------------------------------------------------------------')
print(misnomdifa, tot)
import statistics as stcs

print(stcs.mean(rep),len(rep))

In [ ]:
col3='af'
col1='ri'
e=0
r=0
o=0
list1=[]
rep=[]
misnomdifa=0
for x in ndf1.index:
    if not ndf1[col1][x] in list1 and ndf1[col1][x] != 0:
        n=ndf1[col1][ndf1[col1]==ndf1[col1][x]].count()
        list1.append(ndf1[col1][x])
        rep.append(n)
        e=0
        r=0
        o=0
        for y in ndf1[col1][ndf1[col1]==ndf1[col1][x]].index:
            if ndf1[col3][x] != 0 and ndf1[col3][y] != 0:
                if ndf1[col3][x] != ndf1[col3][y]:
                    r+=1
                    print(ndf1[col3][x],ndf1[col3][y])
    if e != 0 or r != 0 or o != 0:
        print(ndf1[col1][x],n)
        print('H:',r)
        misnomdifa+=1
        print('----------------------------------------------------------------------------------')
print(misnomdifa)

In [ ]:
col3='af'
col1='oi'

list1=[]
rep=[]
misnomdifa=0
for x in ndf1.index:
    if not ndf1[col1][x] in list1 and ndf1[col1][x] != 0:
        n=ndf1[col1][ndf1[col1]==ndf1[col1][x]].count()
        list1.append(ndf1[col1][x])
        rep.append(n)
        e=0
        r=0
        o=0
        for y in ndf1[col1][ndf1[col1]==ndf1[col1][x]].index:
            if ndf1[col3][x] != 0 and ndf1[col3][y] != 0:
                if ndf1[col3][x] != ndf1[col3][y]:
                    r+=1
                    print(ndf1[col3][x],ndf1[col3][y])
    if e != 0 or r != 0 or o != 0:
        print(ndf1[col1][x],n)
        print('H:',r)
        misnomdifa+=1
        print('----------------------------------------------------------------------------------')
print(misnomdifa)

In [ ]:
def f(x):
    fac=1
    for i in range(x):
        fac=fac*(i+1)
    return fac

nac=len(ndf1.index)-13500

f(nac)/(2*f(nac-2))

# Vectorización

In [ ]:
charac=[]
for x in ndf1.index:
    for y in ndf1.columns:
        try:
            for z in ndf1[y][x]:
                for w in z:
                    if not w in charac:
                        charac.append(w)
        except Exception:
            pass
characters = 'ñ'.join(sorted(charac))
key=characters.split('ñ')
length     = len(key)
print(key,'\n',length)

In [37]:
noe= {}
for x in ndf1.columns:
    noe[x]=[]
    for y in ndf1.index:
        if type(ndf1[x][y]) == list:
            noe[x].append(len(ndf1[x][y]))
        else:
            noe[x].append(1)

In [38]:
noedf=pd.DataFrame(noe)

In [ ]:
sns.countplot(x = 'au', data = noedf)

In [ ]:
plt.figure(i)
sns.boxplot(x = 'au', data = noedf)

In [ ]:
sns.countplot(x = 'de', data = noedf)

In [ ]:
plt.figure(i)
sns.boxplot(x = 'de', data = noedf)

In [ ]:
nukey=noedf.describe()
nukey['au']['max']=10
nukey['de']['max']=10
nukey

In [ ]:
for x in ndf1.columns:
    print(x,ndf1[x][3490])

In [ ]:
for x in ndf1.columns:
    print(x,ndf1[x][4501])

In [ ]:
key='0 1 2 3 4 5 6 7 8 9 a b c d e f g h i j k l m n o p q r s t u v w x y z'
key=key.split(' ')

numofrows=0
columnsta=[]
for x in ndf1.columns:
    if not x in ['em','ri','oi']:
        columnsta.append(x)
        numofrows=numofrows+nukey[x]['max']
numofcols=len(key)

img=[]
img1=[]
au1=3490
for x in ndf1.columns:
    if type(ndf1[x][au1]) == list:
        currentlist=ndf1[x][au1].copy()
        random.shuffle(currentlist)
        for y in range(int(nukey[x]['max'])):
            newl=[0]*numofcols
            try:
                for z in currentlist[y]:
                    if z in key:
                        newl[key.index(z)]+=1
            except Exception:
                pass
            img1.append(newl)
    elif type(ndf1[x][au1]) == int:
        for y in range(int(nukey[x]['max'])):
            newl=[0]*numofcols
            img1.append(newl)
    elif type(ndf1[x][au1]) == str:
        for y in range(int(nukey[x]['max'])):
            newl=[0]*numofcols
            if y == 0:
                for z in ndf1[x][au1]:
                    if z in key:
                        newl[key.index(z)]+=1
            img1.append(newl)
img2=[]
au2=4501
for x in ndf1.columns:
    if type(ndf1[x][au2]) == list:
        currentlist=ndf1[x][au2].copy()
        random.shuffle(currentlist)
        for y in range(int(nukey[x]['max'])):
            newl=[0]*numofcols
            try:
                for z in currentlist[y]:
                    if z in key:
                        newl[key.index(z)]+=1
            except Exception:
                pass
            img2.append(newl)
    elif type(ndf1[x][au2]) == int:
        for y in range(int(nukey[x]['max'])):
            newl=[0]*numofcols
            img2.append(newl)
    elif type(ndf1[x][au2]) == str:
        for y in range(int(nukey[x]['max'])):
            newl=[0]*numofcols
            if y == 0:
                for z in ndf1[x][au2]:
                    if z in key:
                        newl[key.index(z)]+=1
            img2.append(newl)
for i in range(int(numofrows)):
    img.append(img1[i]+img2[i])

for x in img:
    print(x)

In [ ]:
def f(x):
    fac=1
    for i in range(x):
        fac=fac*(i+1)
    return fac

col1='af'
aualch=[]
totalof=0
for x in ndf1[col1]:
    if not x in aualch:
        n=ndf1[col1][ndf1[col1]==x].count()
        if n != 1:
            totalof=totalof+(f(n)/(2*f(n-2)))
            print(x,n)
        aualch.append(x)
print(totalof)

In [48]:
col1='af'
col2='em'
col3='ri'
col4='oi'
list1=[]
sameau=[]
for x in ndf1.index:
    if not ndf1[col1][x] in list1:
        if ndf1[col1][ndf1[col1]==ndf1[col1][x]].count() != 1:
            currentlist=ndf1[col1][ndf1[col1]==ndf1[col1][x]].index
            for y in range(len(currentlist)):
                for z in range(y+1,len(currentlist)):
                    e=0
                    r=0
                    o=0
                    au1=currentlist[y]
                    au2=currentlist[z]
                    if ndf1[col2][au1] != 0 and ndf1[col2][au2] != 0:
                        if ndf1[col2][au1][1] == ndf1[col2][au2][1]:
                            if ndf1[col2][au1][0] == ndf1[col2][au2][0]:
                                e+=1
                            else:
                                e=e-1
                    if ndf1[col3][au1] != 0 and ndf1[col3][au2] != 0:
                        if ndf1[col3][au1] == ndf1[col3][au2]:
                            r+=1
                        else:
                            r=r-1
                    if ndf1[col4][au1] != 0 and ndf1[col4][au2] != 0:
                        if ndf1[col4][au1] == ndf1[col4][au2]:
                            o+=1
                        else:
                            o=o-1
                    if e+r+o > 0:
                        sameau.append([[au1,au2],1])
        list1.append(ndf1[col1][x])

In [49]:
col1='af'
col2='em'
col3='ri'
col4='oi'
list2=[]
diffau=[]
for x in range(len(ndf1.index)):
    con=0
    for y in range(x+1,len(ndf1.index)):
        if con < 3:
            o=0
            au1=ndf1.index[x]
            try:
                au2=ndf1.index[y+29]
            except Exception:
                break
            if ndf1[col4][au1] != 0 and ndf1[col4][au2] != 0:
                if ndf1[col4][au1] == ndf1[col4][au2]:
                    o+=1
                else:
                    o=o-1
            if o < 0:
                con+=1
                diffau.append([[au1,au2],0])
        else:
            break

In [50]:
cdiffau=diffau.copy()
random.shuffle(cdiffau)
differentau=cdiffau[:6477]

In [51]:
random.shuffle(sameau)

In [52]:
def vecto(au1,au2):
    import random
    
    key='0 1 2 3 4 5 6 7 8 9 a b c d e f g h i j k l m n o p q r s t u v w x y z'
    key=key.split(' ')

    numofrows=0
    columnsta=[]
    for x in ndf1.columns:
        if not x in ['em','ri','oi']:
            columnsta.append(x)
            numofrows=numofrows+nukey[x]['max']
    numofcols=len(key)
    img=[]
    img1=[]
    img2=[]
    for x in columnsta:
        if type(ndf1[x][au1]) == list:
            currentlist=ndf1[x][au1].copy()
            random.shuffle(currentlist)
            for y in range(int(nukey[x]['max'])):
                newl=[0]*numofcols
                try:
                    for z in currentlist[y]:
                        if z in key:
                            newl[key.index(z)]+=1
                except Exception:
                    pass
                img1.append(newl)
        elif type(ndf1[x][au1]) == int:
            for y in range(int(nukey[x]['max'])):
                newl=[0]*numofcols
                img1.append(newl)
        elif type(ndf1[x][au1]) == str:
            for y in range(int(nukey[x]['max'])):
                newl=[0]*numofcols
                if y == 0:
                    for z in ndf1[x][au1]:
                        if z in key:
                            newl[key.index(z)]+=1
                img1.append(newl)
    for x in columnsta:
        if type(ndf1[x][au2]) == list:
            currentlist=ndf1[x][au2].copy()
            random.shuffle(currentlist)
            for y in range(int(nukey[x]['max'])):
                newl=[0]*numofcols
                try:
                    for z in currentlist[y]:
                        if z in key:
                            newl[key.index(z)]+=1
                except Exception:
                    pass
                img2.append(newl)
        elif type(ndf1[x][au2]) == int:
            for y in range(int(nukey[x]['max'])):
                newl=[0]*numofcols
                img2.append(newl)
        elif type(ndf1[x][au2]) == str:
            for y in range(int(nukey[x]['max'])):
                newl=[0]*numofcols
                if y == 0:
                    for z in ndf1[x][au2]:
                        if z in key:
                            newl[key.index(z)]+=1
                img2.append(newl)
    for i in range(int(numofrows)):
        img.append(img1[i]+img2[i])

    return img

In [ ]:
nndata=[]
for x in sameau:
    nndata.append([vecto(x[0][0],x[0][1]),[x[0][0],x[0][1]],x[1]])
for x in differentau:
    nndata.append([vecto(x[0][0],x[0][1]),[x[0][0],x[0][1]],x[1]])

for x in nndata:
    for y in x[0]:
        print(y)
    for y in ndf1.columns:
        print(ndf1[y][x[1][0]])
    print('-------------------------------------------------------------------------------------------------------')
    for y in ndf1.columns:
        print(ndf1[y][x[1][1]])
    print('-------------------------------------------------------------------------------------------------------')
    print(x[2])
    print('llllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllll')
    

In [54]:
nndata_train=[]
for x in sameau[:5000]:
    nndata_train.append([vecto(x[0][0],x[0][1]),x[1]])
for x in differentau[:5000]:
    nndata_train.append([vecto(x[0][0],x[0][1]),x[1]])

nndata_test=[]
for x in sameau[5000:]:
    nndata_test.append([vecto(x[0][0],x[0][1]),x[1],[x[0][0],x[0][1]]])
for x in differentau[5000:]:
    nndata_test.append([vecto(x[0][0],x[0][1]),x[1],[x[0][0],x[0][1]]])

# Red Neuronal


In [55]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

from tensorboard.plugins.hparams import api as hp
import tensorflow as tf
from collections import OrderedDict, namedtuple
from itertools import product

from tensorflow.keras.layers import Input, Dense, Activation, Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from datetime import datetime
from matplotlib.pyplot import imshow

import pickle
import time

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import matplotlib

from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import seaborn as sns

%matplotlib inline
np.random.seed(1)

In [56]:
random.shuffle(nndata_train)
random.shuffle(nndata_test)

In [57]:
x_te, y_te= [], []
for x in nndata_test:
    x_te.append(x[0])
    y_te.append(x[1])

In [58]:
x_tr, y_tr= [], []
for x in nndata_train:
    x_tr.append(x[0])
    y_tr.append(x[1])

In [59]:
x_te = np.array(x_te)
y_te = np.array(y_te)
x_tr = np.array(x_tr)
y_tr = np.array(y_tr)

In [ ]:
x_te.shape, y_te.shape

In [ ]:
x_tr.shape, y_tr.shape

In [ ]:
df = pd.DataFrame(data = y_tr,
                  columns = ['label'])
df['label'].value_counts()

In [ ]:
sns.countplot(x = 'label', data = df, width=0.8)

In [64]:
dic = {0: 'different author', 1: 'same author'}

train_y = y_tr
test_y = y_te

In [65]:
x_train = x_tr.astype('float32') / 31.0
x_test = x_te.astype('float32') / 31.0

In [66]:
def architecture_o(input_shape, num_classes, activation, dropout=False, batch_normalization=False):

    model = Sequential()

    model.add(Conv2D(filters=16,
                     kernel_size=2,
                     padding='same',
                     kernel_initializer='glorot_uniform',
                     bias_initializer='zeros',
                     input_shape=input_shape))

    model.add(Activation(activation))
    if batch_normalization:
        model.add(BatchNormalization())

    model.add(Conv2D(filters=32,kernel_size=2,padding='same',kernel_initializer='glorot_uniform',bias_initializer='zeros'))
    model.add(Activation(activation))
    if batch_normalization:
        model.add(BatchNormalization())

    model.add(MaxPooling2D())

    model.add(Conv2D(64, 2, padding = 'same', kernel_initializer = 'glorot_uniform',bias_initializer='zeros'))
    model.add(Activation(activation))
    if batch_normalization:
        model.add(BatchNormalization())

    model.add(MaxPooling2D())

    model.add(Flatten())
    if dropout:
        model.add(Dropout(0.50))

    model.add(Dense(128, kernel_initializer='glorot_uniform',bias_initializer='zeros'))
    model.add(Activation(activation))
    if batch_normalization:
        model.add(BatchNormalization())
    if dropout:
        model.add(Dropout(0.50))

    model.add(Dense(num_classes))
    model.add(Activation('sigmoid',name='Model'))

    return model

In [67]:
one_image = (int(numofrows), numofcols*2, 1)
activation='sigmoid'
num_classes = 1
dropout=True
batch_normalization=True

model = architecture_o(one_image,num_classes,activation,dropout,batch_normalization)

In [ ]:
plot_model(model, to_file='model.png', show_shapes=True, rankdir='TB', show_layer_names=True)

In [69]:
learning_rate = 0.01

optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.0, nesterov=False)

In [70]:
loss_function = 'mse'
metric_function = 'accuracy'

In [71]:
model.compile(optimizer = optimizer, loss = loss_function, metrics = [metric_function])

In [ ]:
start_time = time.time()

validation_portion = 0.1
batch_size=128
num_epochs = 500

callback = EarlyStopping(monitor="val_loss", min_delta=0.001, patience=10, restore_best_weights=True, start_from_epoch=40)

history = model.fit(x=x_train, y=train_y, epochs=num_epochs, batch_size=batch_size, \
                            validation_split=validation_portion, shuffle=True,
                            callbacks=[callback], verbose=1)

end_time = time.time()
print("Time for training: {:10.4f}s".format(end_time - start_time))

In [ ]:
plt.figure(figsize=(8,6))

plt.plot(history.history['loss'], color='red')
plt.plot(history.history['val_loss'], color='blue')
plt.title('Lr = 0.01, Dropout = 0.50')
plt.ylabel('Cost', size=16)
plt.xlabel('Epoch',size=16)
plt.legend(['Train', 'Validation'], loc='upper right', fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))

plt.plot(history.history['accuracy'], color='red')
plt.plot(history.history['val_accuracy'], color='blue')
plt.title('Lr = 0.01, Dropout = 0.50')
plt.ylabel('Accuracy', size=16)
plt.xlabel('Epoch', size=16)
plt.legend(['Train', 'Validation'], loc='lower right', fontsize=16)
plt.show()

In [ ]:
evaluations = model.evaluate(x = x_test, y = test_y)

print ("Loss = " + str(evaluations[0]))
print ("Test Accuracy = " + str(evaluations[1]))

In [ ]:
predictions = model.predict(x_test)
sample = 3
prediction = predictions[sample]

print('Para el par', sample, 'la predicción es', prediction)
print('El par', sample, 'son', test_y[sample])

# Evaluación del modelo

In [ ]:
abserr=[]
for x in range(len(predictions)):
    if abs(test_y[x]-predictions[x][0]) > 1-evaluations[0]:
        abserr.append(x)
len(abserr)

In [ ]:
for x in abserr:
    i1=nndata_test[x][2][0]
    i2=nndata_test[x][2][1]
    print(test_y[x],predictions[x][0])
    print('-------------------------------------------------------------------')
    for y in ndf1.columns:
        print(y,ndf1[y][i1])
    print('-------------------------------------------------------------------')
    for y in ndf1.columns:
        print(y,ndf1[y][i2])
    print('======================================================================================================================================')